In [2]:
#imports
import yaml
import cv2
import os
import numpy as np

In [5]:

yaml_file_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/dataset.yaml'

with open(yaml_file_path, 'r') as file:
    dataset_info = yaml.safe_load(file)

image_width, image_height = 224, 224

def read_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_width, image_height))
    image = image / 255.0  #norm
    return image

train_images = []
train_labels = []

train_path = os.path.join(dataset_info['path'], dataset_info['train'])
for class_name, class_id in dataset_info['names'].items():
    class_path = train_path
    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        
        label = 0
        
        image = read_and_preprocess_image(image_path)
        
        train_images.append(image)
        train_labels.append(label)

X_train = np.array(train_images)
y_train = np.array(train_labels)
